In [3]:
import pickle
import json
import seaborn as sns
import matplotlib.transforms as mtransforms
from bread import vis
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bread.algo import tracking
from bread.data import *
from sklearn.metrics import confusion_matrix
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import sys
import os.path
sys.path.append('../../src/libraries')  # import YeaZ

In [22]:
seg = SegmentationFile.from_h5(
    '../../data/external/external_segmentation.h5').get_segmentation("FOV0")
microscopy = Microscopy.from_tiff(
    '../../data/external/external_microscopy.tif')
# first make fatures with scale 1 to find the correct scale
feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)

# on average, first frame cells have an area of 2078.0 in the train set
mean_cell_area = 2078.0
area_list = []
cells_first_frame = seg.cell_ids(0)
for cell in cells_first_frame:
    area_list.append(feat.cell_area(0, cell))
scale = mean_cell_area / np.mean(area_list)
print(scale)
print(np.mean(area_list))
print(np.mean(area_list) * scale)
# replace features with correct scale
feat = Features(seg, nn_threshold=12, scale_length=scale, scale_time=1)

2.88410825815406
720.5
2078.0


In [4]:
res_external = pd.read_csv(
    '/Users/farzanehwork/Documents/codes/bread/data/generated/tracking/compare/colony_external/results_2023-07-04_20-43-11.csv')

In [11]:
yeaz_errors = res_external.loc[(res_external['method'] == 'yeaz') & (
    res_external['fp']+res_external['fn'] > 0) & (res_external.t2-res_external.t1 < 2)]
yeaz_errors

,Unnamed: 0,t1,t2,method,tp,fp,tn,fn,acc,f1,num_cells1,num_cells2,timediff
858,858,97,98,yeaz,125,6,17810,6,0.999331,0.954198,131,137,5
866,866,99,100,yeaz,143,1,21167,1,0.999906,0.993056,144,148,5
894,894,106,107,yeaz,189,2,38007,2,0.999895,0.989529,191,200,5
902,902,108,109,yeaz,195,9,43239,9,0.999586,0.955882,204,213,5
914,914,111,112,yeaz,216,13,54031,13,0.999521,0.943231,229,237,5
918,918,112,113,yeaz,214,23,59227,23,0.999227,0.902954,237,251,5


In [12]:
import YeaZ.unet.hungarian as YeaZ_hung

In [13]:
import scipy.spatial.distance


def match_cellids_from_segs(segframe1: np.ndarray, segframe2: np.ndarray, eps=1e-6):
    # Matches cellids between two identical segmentations, which use different numberings
    seg1 = Segmentation(segframe1[None, ...], fov='FOV0')
    seg2 = Segmentation(segframe2[None, ...], fov='FOV0')
    cms1 = seg1.cms(0)
    cms2 = seg2.cms(0)

    # compare centers of masses
    dists = scipy.spatial.distance.cdist(cms1, cms2)
    matches = dists < eps
    idx1, idx2 = matches.nonzero()
    assert not len(idx2) < len(cms1), 'not all cells were matched'
    assert not len(idx2) > len(cms1), 'too many cells were matched'

    return idx2

In [24]:
for i, row in yeaz_errors.iterrows():
    idt1, idt2 = row.t1, row.t2
    seg_hat = YeaZ_hung.correspondence(seg[idt1], seg[idt2])
    idmap2 = match_cellids_from_segs(seg[idt2], seg_hat)

    y = tracking.assignment_from_segmentation(seg, idt1, idt2)
    yhat = tracking.assignment_from_segmentation(seg, idt1, idt2)[
        :, idmap2]

    print(confusion_matrix(y.flatten(), yhat.flatten()))
    print(idt1, idt2)

    cellids1, cellids2 = seg.cell_ids(row.t1), seg.cell_ids(row.t2)
    # coloriter = iter(plt.rcParams['axes.prop_cycle'].by_key()['color'])
    for idx1, (idx2, idx2_hat) in enumerate(zip(np.argmax(y, axis=1), np.argmax(yhat, axis=1))):
        if idx2 != idx2_hat:
            # color = next(coloriter)
            print(
                f'Expected {cellids1[idx1]}->{cellids2[idx2]}, got {cellids1[idx1]}->{cellids2[idx2_hat]}')
        # else:
            # print(f'Correct {cellids1[idx1]}->{cellids2[idx2]}')
    #         vis._vis._plot_ellipse(feat._ellipse(cellids1[idx1], row.t1), figax=(
    #             fig, axs[0]), color=color, linewidth=2, alpha=0.8, draw_axes=False)
    #         vis._vis._plot_ellipse(feat._ellipse(cellids2[idx2], row.t2), figax=(
    #             fig, axs[1]), color=color, linewidth=2, alpha=0.8, draw_axes=False)
    #         vis._vis._plot_ellipse(feat._ellipse(cellids2[idx2_hat], row.t2), figax=(
    #             fig, axs[1]), color=color, linestyle='--', linewidth=2, alpha=0.8, draw_axes=False)
    # plt.show()

[[17810     6]
 [    6   125]]
97 98
Expected 127->127, got 127->128
Expected 128->128, got 128->129
Expected 129->129, got 129->130
Expected 130->130, got 130->131
Expected 131->131, got 131->132
Expected 132->132, got 132->126
[[21167     1]
 [    1   143]]
99 100
Expected 143->143, got 143->148
[[38007     2]
 [    2   189]]
106 107
Expected 191->191, got 191->192
Expected 192->192, got 192->190
[[43239     9]
 [    9   195]]
108 109
Expected 197->197, got 197->198
Expected 198->198, got 198->199
Expected 199->199, got 199->200
Expected 200->200, got 200->201
Expected 201->201, got 201->202
Expected 202->202, got 202->203
Expected 203->203, got 203->204
Expected 204->204, got 204->205
Expected 205->205, got 205->196
[[54031    13]
 [   13   216]]
111 112
Expected 219->219, got 219->220
Expected 220->220, got 220->221
Expected 221->221, got 221->222
Expected 222->222, got 222->223
Expected 223->223, got 223->224
Expected 224->224, got 224->225
Expected 225->225, got 225->226
Expected